# Preprocessing & feature Engineering

## 1. Preprocessing 
- Data shape
- Column prerocessing

## 2. Feature/Data Transformation
- Outlier  
- New feature

## 3. Missing Data Imputation
- Regression
- Mean

## 4. Dimensionality Reduction
- Features with Bad or Constant Data
- Multicollinearity and Variance Inflation Factor

In [1]:
%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings("ignore")
import sys
import datetime
import scipy as sp
import statsmodels.stats.api as sms
import statsmodels.api as sm
from patsy import dmatrix
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
import utils.preprocessing as pp 
import utils.correlation as cr
import utils.statsmodel_helper as st

df_macro = pd.read_csv('../code/data/macro.csv', parse_dates=['timestamp'])
df_train = pd.read_csv('../code/data/train.csv', index_col=0, parse_dates=['timestamp'])
df_test = pd.read_csv('../code/data/test.csv', index_col=0, parse_dates=['timestamp'])

## 1. Preprocessing 
- Data merge
- Column prerocessing

### Data merge

In [2]:
df_train.shape

(30471, 291)

In [3]:
df_train.tail(1)

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,c

In [4]:
df_test.shape

(7662, 290)

In [5]:
df_test.tail(1)

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,c

In [ ]:
# df_train_macro = df_train.merge(df_macro, left_on='timestamp', right_on='timestamp', how='left').set_index(df_train.index)
# df_test_macro = df_test.merge(df_macro, left_on='timestamp', right_on='timestamp', how='left').set_index(df_test.index)

### column preprocessing

In [6]:
# macro data에서 missing value 50% 이상이면 drop 
del df_macro["modern_education_share"]
del df_macro["old_education_build_share"]
del df_macro["provision_retail_space_sqm"]
del df_macro["provision_retail_space_modern_sqm"]
del df_macro["theaters_viewers_per_1000_cap"]
del df_macro["museum_visitis_per_100_cap"]
# odd number 45,34 etc
del df_macro["child_on_acc_pre_school"]

# material은 카테고리
df_train.loc[df_train['material'].isnull(), 'material'] = 0
df_train['material'] = df_train['material'].replace([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0], ['a', 'b', 'c', 'd', 'e', 'f', 'e'])
df_test['material'] = df_test['material'].replace([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0], ['a', 'b', 'c', 'd', 'e', 'f', 'e'])

# column rename
df_train.rename(columns={'build_count_1921-1945': 'build_count_1921_1945', 'build_count_1946-1970': 'build_count_1946_1970', 'build_count_1971-1995': 'build_count_1971_1995'}, inplace=True)
df_test.rename(columns={'build_count_1921-1945': 'build_count_1921_1945', 'build_count_1946-1970': 'build_count_1946_1970', 'build_count_1971-1995': 'build_count_1971_1995'}, inplace=True)

## 2. Feature/Data Transformation
- Outlier  
- New feature

## Outlier

In [7]:
# Drop data with extremely big price #
df_train = df_train.drop([2121]) 

# Replace outliers with proper value #
df_train.loc[df_train.state == 33, 'state'] = 3
df_train.loc[df_train['life_sq'] > 1000,     'life_sq']       = np.median(df_train['life_sq'].dropna())
df_train.loc[df_train['kitch_sq'] > 250,     'kitch_sq']      = np.median(df_train['kitch_sq'].dropna())
df_train.loc[df_train['num_room'] > 6,       'num_room']      = np.median(df_train['num_room'].dropna())
df_train.loc[df_train['build_year'] > 2017,  'build_year']    = np.median(df_train['build_year'].dropna())
df_train.loc[df_train['build_year'] < 1800,  'build_year']    = np.median(df_train['build_year'].dropna())
df_train.loc[df_train['floor'] > 50,         'floor']         = np.median(df_train['floor'].dropna())
df_train.loc[df_train['max_floor'] > 60,     'max_floor']     = np.median(df_train['max_floor'].dropna())
df_train.loc[df_train.full_sq == 0, 'full_sq'] = 50
df_train = df_train[df_train.price_doc/df_train.full_sq <= 600000]
df_train = df_train[df_train.price_doc/df_train.full_sq >= 10000]

df_test.loc[df_test['life_sq'] > 1000,     'life_sq']       = np.median(df_test['life_sq'].dropna())
df_test.loc[df_test['kitch_sq'] > 250,     'kitch_sq']      = np.median(df_test['kitch_sq'].dropna())
df_test.loc[df_test['num_room'] > 6,       'num_room']      = np.median(df_test['num_room'].dropna())
df_test.loc[df_test['build_year'] > 2017,  'build_year']    = np.median(df_test['build_year'].dropna())
df_test.loc[df_test['build_year'] < 1800,  'build_year']    = np.median(df_test['build_year'].dropna())
df_test.loc[df_test['floor'] > 50,         'floor']         = np.median(df_test['floor'].dropna())
df_test.loc[df_test['max_floor'] > 60,     'max_floor']     = np.median(df_test['max_floor'].dropna())
df_test.loc[df_test.full_sq == 0, 'full_sq'] = 50

## New features

In [8]:
# Add month and day of week #
df_train['month'] = df_train.timestamp.dt.month
df_train['dow'] = df_train.timestamp.dt.dayofweek

df_test['month'] = df_test.timestamp.dt.month
df_test['dow'] = df_test.timestamp.dt.dayofweek

# Create new features that might help #
df_train['rel_floor'] = df_train['floor'] / df_train['max_floor'].astype(float)
df_train['rel_kitch_sq'] = df_train['kitch_sq'] / df_train['full_sq'].astype(float)

df_test['rel_floor'] = df_test['floor'] / df_test['max_floor'].astype(float)
df_test['rel_kitch_sq'] = df_test['kitch_sq'] / df_test['full_sq'].astype(float)

df_train.apartment_name=df_train.sub_area + df_train['metro_km_avto'].astype(str)
df_test.apartment_name=df_test.sub_area + df_test['metro_km_avto'].astype(str)

df_train['room_size'] = df_train['life_sq'] / df_train['num_room'].astype(float)
df_test['room_size'] = df_test['life_sq'] / df_test['num_room'].astype(float)

# Average price corresponding to sub_area and ID_* #
id_features = ['ID_metro',
    'ID_railroad_station_walk', \
    'ID_big_road1', \
    'ID_big_road2', \
    'ID_railroad_terminal', \
    'ID_bus_terminal']

for id_f in id_features:
    df_test['avg_price_' + id_f] = 0.0
    for val in df_test[id_f].unique():
        if val == 171 and id_f == 'ID_metro':
            df_test.loc[df_test.ID_metro == 171, 'avg_price_ID_metro'] = df_train_macro[df_train.ID_metro == 170]['price_doc'].mean()
            continue
        if val == 132 and id_f == 'ID_railroad_station_walk':
            df_test.loc[df_test.ID_railroad_station_walk == 132, 'avg_price_ID_railroad_station_walk'] = df_train[df_train.ID_railroad_station_walk == 131]['price_doc'].mean()
            continue
        if val == 121 and id_f == 'ID_railroad_station_walk':
            df_test.loc[df_test.ID_railroad_station_walk == 122, 'avg_price_ID_railroad_station_walk'] = df_train[df_train.ID_railroad_station_walk == 131]['price_doc'].mean()
            continue
        avg = df_train[df_train[id_f] == val]['price_doc'].mean()
        df_test.loc[df_test[id_f] == val, 'avg_price_' + id_f] = avg
    del df_test[id_f]
    
for id_f in id_features:
    df_train['avg_price_' + id_f] = 0.0
    for val in df_train[id_f].unique():
        avg = df_train[df_train[id_f] == val]['price_doc'].mean()
        df_train.loc[df_train[id_f] == val, 'avg_price_' + id_f] = avg
    del df_train[id_f]
    
cols = list(df_train.columns.values)
cols.pop(cols.index('price_doc'))
df_train = df_train[cols + ['price_doc']]


df_test['avg_price_sub_area'] = 0.0
df_train['avg_price_sub_area'] = 0.0
for subarea in df_train['sub_area'].unique():
    avg = df_train[df_train['sub_area'] == subarea]['price_doc'].mean()
    df_train.loc[df_train['sub_area'] == subarea, 'avg_price_sub_area'] = avg
    df_test.loc[df_test['sub_area'] == subarea, 'avg_price_sub_area'] = avg
del df_train['sub_area']
del df_test['sub_area']

## 3. Missing Data Imputation
- Regression
- Mean

### Regression

In [9]:
pp.find_missing_data_columns(df_train)

,missing_column,missing_count
2,life_sq,6379
3,floor,167
4,max_floor,9540
6,build_year,13570
7,num_room,9540
8,kitch_sq,9540
9,state,13526
16,preschool_quota,6677
19,school_quota,6674
22,hospital_beds_raion,14415


In [10]:
pp.find_missing_data_columns(df_test)

,missing_column,missing_count
2,life_sq,1176
6,build_year,1049
9,state,694
10,product_type,33
16,preschool_quota,1596
19,school_quota,1595
22,hospital_beds_raion,3418
66,raion_build_count_with_material_info,1218
67,build_count_block,1218
68,build_count_wood,1218


In [11]:
pp.find_missing_data_columns(df_macro)

,missing_column,missing_count
2,gdp_quart,90
3,gdp_quart_growth,90
4,cpi,31
5,ppi,31
6,gdp_deflator,365
7,balance_trade,31
8,balance_trade_growth,90
9,usdrub,3
10,eurrub,3
11,brent,3


In [13]:
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_macro.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train._get_numeric_data()[df_train._get_numeric_data() < 0] = 0
df_test._get_numeric_data()[df_test._get_numeric_data() < 0] = 0
df_macro._get_numeric_data()[df_macro._get_numeric_data() < 0] = 0

In [14]:
pp.impute_by_regression(df_train, 4, 0.3)
pp.impute_by_regression2(df_test, 4, 0.3)
pp.impute_by_regression2(df_macro, 4, 0.3)
# imputation
pp.impute_num_mode(df_train)
pp.impute_num_mode(df_test)
pp.impute_num_mode(df_macro)
# pp.imput_cat_mode(df_train_macro)
# pp.imput_cat_mode(df_test_macro)

In [15]:
pp.find_missing_data_columns(df_train)

,missing_column,missing_count


In [16]:
pp.find_missing_data_columns(df_test)

,missing_column,missing_count
10,product_type,33


In [17]:
pp.find_missing_data_columns(df_macro)

,missing_column,missing_count


# 4. Dimensionality Reduction

## Features with Bad or Constant Data

In [18]:
# Features with high correlation with other #
features_to_remove = [
    'children_preschool', 'children_school', 'male_f', \
    'female_f', 'young_male', 'young_female', 'work_male', \
    'work_female', 'ekder_male', 'ekder_female',\
    '0_6_all', '0_6_male', '0_6_female',\
    '7_14_all', '7_14_male', '7_14_female', '0_17_male', '0_17_female',\
    '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female',\
    'metro_km_walk', 'railroad_station_walk_km',\
    'railroad_station_avto_km', 'public_transport_station_km' \
]
for f in features_to_remove:
    del df_train[f]
    del df_test[f]

In [19]:
# Constant features #
consts = [col for col in df_train.columns if len(df_train[col].value_counts().index) == 1]
for const in consts:
    del df_train[const]
    del df_test[const]

## Multicollinearity and Variance Inflation Factor

In [20]:
# Low correlation with price #
corr_limit = 0.1
for column in df_train._get_numeric_data().columns.drop('price_doc').values:
    if abs(df_train[column].corr(df_train['price_doc'])) < corr_limit:
        df_train = df_train.drop(column, axis=1)
        if column in df_test.columns.values:
            df_test = df_test.drop(column, axis=1)

In [21]:
# Calculate VIF #
df_train[df_train==np.inf]=np.nan
df_train.fillna(df_train.mean(), inplace=True)
categorial_ivs = set(df_train.columns.drop('timestamp')) - set(df_train._get_numeric_data().columns)
numeric_ivs = df_train._get_numeric_data().columns.drop('price_doc')
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    df_train[numeric_ivs].values, i) for i in range(df_train[numeric_ivs].shape[1])]
vif["features"] = df_train[numeric_ivs].columns

In [22]:
list(vif.loc[vif['VIF Factor'] == np.inf].features)

['raion_popul',
 'young_all',
 'work_all',
 'ekder_all',
 'cafe_count_3000',
 'cafe_count_3000_na_price',
 'cafe_count_3000_price_500',
 'cafe_count_3000_price_1000',
 'cafe_count_3000_price_1500',
 'cafe_count_3000_price_2500',
 'cafe_count_3000_price_4000',
 'cafe_count_3000_price_high',
 'cafe_count_5000',
 'cafe_count_5000_na_price',
 'cafe_count_5000_price_500',
 'cafe_count_5000_price_1000',
 'cafe_count_5000_price_1500',
 'cafe_count_5000_price_2500',
 'cafe_count_5000_price_4000',
 'cafe_count_5000_price_high']

In [25]:
# Calculate VIF #
df_macro[df_macro==np.inf]=np.nan
df_macro.fillna(df_macro.mean(), inplace=True)
categorial_ivs = set(df_macro.columns.drop('timestamp')) - set(df_macro._get_numeric_data().columns)
numeric_ivs = df_macro._get_numeric_data().columns
vif2 = pd.DataFrame()
vif2["VIF Factor"] = [variance_inflation_factor(
    df_macro[numeric_ivs].values, i) for i in range(df_macro[numeric_ivs].shape[1])]
vif2["features"] = df_macro[numeric_ivs].columns

In [28]:
list(vif2.loc[vif2['VIF Factor'] == np.inf].features)

['gdp_deflator',
 'gdp_annual',
 'gdp_annual_growth',
 'grp',
 'grp_growth',
 'real_dispos_income_per_cap_growth',
 'salary',
 'salary_growth',
 'retail_trade_turnover',
 'retail_trade_turnover_per_cap',
 'retail_trade_turnover_growth',
 'labor_force',
 'unemployment',
 'employment',
 'invest_fixed_capital_per_cap',
 'invest_fixed_assets',
 'profitable_enterpr_share',
 'unprofitable_enterpr_share',
 'share_own_revenues',
 'overdue_wages_per_cap',
 'fin_res_per_cap',
 'marriages_per_1000_cap',
 'divorce_rate',
 'construction_value',
 'invest_fixed_assets_phys',
 'pop_natural_increase',
 'pop_migration',
 'pop_total_inc',
 'childbirth',
 'mortality',
 'housing_fund_sqm',
 'lodging_sqm_per_cap',
 'water_pipes_share',
 'baths_share',
 'sewerage_share',
 'gas_share',
 'hot_water_share',
 'electric_stove_share',
 'heating_share',
 'old_house_share',
 'average_life_exp',
 'infant_mortarity_per_1000_cap',
 'perinatal_mort_per_1000_cap',
 'incidence_population',
 'load_of_teachers_preschool_per

In [29]:
# Features with VIF greater than np.inf
features_to_remove = [
 'raion_popul',
 'young_all',
 'work_all',
 'ekder_all',
 'cafe_count_3000',
 'cafe_count_3000_na_price',
 'cafe_count_3000_price_500',
 'cafe_count_3000_price_1000',
 'cafe_count_3000_price_1500',
 'cafe_count_3000_price_2500',
 'cafe_count_3000_price_4000',
 'cafe_count_3000_price_high',
 'cafe_count_5000',
 'cafe_count_5000_na_price',
 'cafe_count_5000_price_500',
 'cafe_count_5000_price_1000',
 'cafe_count_5000_price_1500',
 'cafe_count_5000_price_2500',
 'cafe_count_5000_price_4000',
 'cafe_count_5000_price_high',
]

for f in features_to_remove:
    if f in df_train:
        del df_train[f]
        del df_test[f]
        
features_to_remove2 = [
 'gdp_deflator',
 'gdp_annual',
 'gdp_annual_growth',
 'grp',
 'grp_growth',
 'real_dispos_income_per_cap_growth',
 'salary',
 'salary_growth',
 'retail_trade_turnover',
 'retail_trade_turnover_per_cap',
 'retail_trade_turnover_growth',
 'labor_force',
 'unemployment',
 'employment',
 'invest_fixed_capital_per_cap',
 'invest_fixed_assets',
 'profitable_enterpr_share',
 'unprofitable_enterpr_share',
 'share_own_revenues',
 'overdue_wages_per_cap',
 'fin_res_per_cap',
 'marriages_per_1000_cap',
 'divorce_rate',
 'construction_value',
 'invest_fixed_assets_phys',
 'pop_natural_increase',
 'pop_migration',
 'pop_total_inc',
 'childbirth',
 'mortality',
 'housing_fund_sqm',
 'lodging_sqm_per_cap',
 'water_pipes_share',
 'baths_share',
 'sewerage_share',
 'gas_share',
 'hot_water_share',
 'electric_stove_share',
 'heating_share',
 'old_house_share',
 'average_life_exp',
 'infant_mortarity_per_1000_cap',
 'perinatal_mort_per_1000_cap',
 'incidence_population',
 'load_of_teachers_preschool_per_teacher',
 'load_of_teachers_school_per_teacher',
 'students_state_oneshift',
 'provision_doctors',
 'provision_nurse',
 'load_on_doctors',
 'power_clinics',
 'hospital_beds_available_per_cap',
 'hospital_bed_occupancy_per_year',
 'turnover_catering_per_cap',
 'seats_theather_rfmin_per_100000_cap',
 'bandwidth_sports',
 'population_reg_sports_share',
 'students_reg_sports_share',
 'apartment_build',
 'apartment_fund_sqm'
]

for f in features_to_remove2:
    if f in df_macro:
        del df_macro[f]

In [30]:
df_train_macro = df_train.merge(df_macro, left_on='timestamp', right_on='timestamp', how='left').set_index(df_train.index)
df_test_macro = df_test.merge(df_macro, left_on='timestamp', right_on='timestamp', how='left').set_index(df_test.index)
cols = list(df_train_macro.columns.values)
cols.pop(cols.index('price_doc'))
df_train_macro = df_train_macro[cols + ['price_doc']]
df_train_macro.to_csv('../code/data/train_macro6.csv', header=True, index=True)
df_test_macro.to_csv('../code/data/test_macro6.csv', header=True, index=True)